In [1]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import pymongo
from pymongo import MongoClient

In [2]:
# Connection to mongo
client = MongoClient('mongodb+srv://<username>:<password>@cluster0.l3pqt.mongodb.net/MSA?retryWrites=true&w=majority')
# Select database
db = client['MSA']
# see list of collections
client.MSA.list_collection_names()

['arima_LaborParticipation_pred_score',
 'CPI_RegionUrban_raw',
 'arima_pop_2019_2024',
 'BusinessApplications_clean',
 'arima_GDP_2019_2024',
 'employment_prediction',
 'Employment_clean',
 'CPI_RegionUrban_clean',
 'population_predicted_2024',
 'employment_clean',
 'LaborForce_clean',
 'GDP_raw',
 'unemployment_ROC_2024',
 'employment_raw',
 'OES_raw',
 'Population_clean',
 'unemployment_predicted_2024',
 'arima_pop_pred_score',
 'msa_codes',
 'arima_unem_pred_score',
 'population_raw',
 'arima_emp_pred_score',
 'LaborForce_raw',
 'arima_cpi_pred_score',
 'UnemploymentRate_clean',
 'arima_emp_2019_2024',
 'arima_annual_wage_pred_score',
 'Employment_raw',
 'arima_2024_ROC_score_total',
 'regions_divisions',
 'arima_business_pred_score',
 'LaborParticipation_clean',
 'unemployment_raw',
 'AnnualMeanWage_clean',
 'GDP_clean',
 'UnemploymentRate_raw',
 'Predicted_2024_ROC_rank_total',
 'arima_GDP_pred_score',
 'Predicted_2024_ROC_rank_total2',
 'GDP_predicted_2024']

In [3]:
# Select the collection with needed data
bus = db.BusinessApplications_clean
df = pd.DataFrame(list(bus.find()))
print(df.dtypes)
df

_id      object
CBSA      int64
2010    float64
2011    float64
2012    float64
2013    float64
2014    float64
2015    float64
2016    float64
2017    float64
2018    float64
2019    float64
dtype: object


,_id,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,603dd31c680af8db371b273a,10180,774.0,818.0,929.0,839.0,978.0,1032.0,1039.0,1053.0,1093.0,1170.0
1,603dd31c680af8db371b273b,10420,4395.0,4610.0,4379.0,4327.0,4354.0,4639.0,4834.0,5175.0,5511.0,5787.0
2,603dd31c680af8db371b273c,10500,923.0,1033.0,1060.0,966.0,1107.0,1041.0,1161.0,1183.0,1300.0,1686.0
3,603dd31c680af8db371b273d,10540,564.0,544.0,537.0,509.0,631.0,608.0,687.0,744.0,843.0,831.0
4,603dd31c680af8db371b273e,10580,4825.0,5284.0,5150.0,5352.0,5216.0,5405.0,6050.0,7728.0,8960.0,7627.0
...,...,...,...,...,...,...,...,...,...,...,...,...
379,603dd31c680af8db371b28b5,49420,924.0,1042.0,908.0,891.0,932.0,974.0,1038.0,1066.0,1213.0,1352.0
380,603dd31c680af8db371b28b6,49620,2167.0,2151.0,2271.0,2123.0,2166.0,2232.0,2365.0,2561.0,2640.0,2756.0
381,603dd31c680af8db371b28b7,49660,2652.0,2808.0,2982.0,2858.0,2890.0,2848.0,2850.0,3144.0,3319.0,3425.0
382,603dd31c680af8db371b28b8,49700,655.0,648.0,614.0,690.0,656.0,715.0,666.0,782.0,892.0,1005.0


In [4]:
df.drop(columns='_id', inplace=True)
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,10180,774.0,818.0,929.0,839.0,978.0,1032.0,1039.0,1053.0,1093.0,1170.0
1,10420,4395.0,4610.0,4379.0,4327.0,4354.0,4639.0,4834.0,5175.0,5511.0,5787.0
2,10500,923.0,1033.0,1060.0,966.0,1107.0,1041.0,1161.0,1183.0,1300.0,1686.0
3,10540,564.0,544.0,537.0,509.0,631.0,608.0,687.0,744.0,843.0,831.0
4,10580,4825.0,5284.0,5150.0,5352.0,5216.0,5405.0,6050.0,7728.0,8960.0,7627.0
...,...,...,...,...,...,...,...,...,...,...,...
379,49420,924.0,1042.0,908.0,891.0,932.0,974.0,1038.0,1066.0,1213.0,1352.0
380,49620,2167.0,2151.0,2271.0,2123.0,2166.0,2232.0,2365.0,2561.0,2640.0,2756.0
381,49660,2652.0,2808.0,2982.0,2858.0,2890.0,2848.0,2850.0,3144.0,3319.0,3425.0
382,49700,655.0,648.0,614.0,690.0,656.0,715.0,666.0,782.0,892.0,1005.0


In [5]:
# for loop to predict 2024 values
prediction2020 = []
prediction2021 = []
prediction2022 = []
prediction2023 = []
prediction2024 = []
for i in range(0,384):
    y = df.iloc[i, 1:].values
    series = pd.Series(y, dtype='int')
    model = ARIMA(series, order=(2, 1, 1))
    model_fit = model.fit()
    pred = model_fit.forecast(5)
    forecast = pred.values.tolist()
    prediction2020.append(forecast[0])
    prediction2021.append(forecast[1])
    prediction2022.append(forecast[2])
    prediction2023.append(forecast[3])
    prediction2024.append(forecast[4])
df['2020'] = prediction2020
df['2021'] = prediction2021
df['2022'] = prediction2022
df['2023'] = prediction2023
df['2024'] = prediction2024

C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:963: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\tsa\statespace\sarimax.py:975: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\Py

C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWa

C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
C:\Users\Joshua\anaconda3\envs\PythonData\lib\site-packages\statsmodels\base\model.py:568: ConvergenceWa

In [6]:
df

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,10180,774.0,818.0,929.0,839.0,978.0,1032.0,1039.0,1053.0,1093.0,1170.0,1190.685108,1238.887634,1273.643132,1314.968775,1353.083222
1,10420,4395.0,4610.0,4379.0,4327.0,4354.0,4639.0,4834.0,5175.0,5511.0,5787.0,5913.494152,6063.475797,6103.425442,6193.935082,6193.053119
2,10500,923.0,1033.0,1060.0,966.0,1107.0,1041.0,1161.0,1183.0,1300.0,1686.0,1749.703930,1811.163761,1872.594293,1934.010857,1995.413561
3,10540,564.0,544.0,537.0,509.0,631.0,608.0,687.0,744.0,843.0,831.0,883.661559,884.031140,909.439471,912.584540,925.190081
4,10580,4825.0,5284.0,5150.0,5352.0,5216.0,5405.0,6050.0,7728.0,8960.0,7627.0,6357.784208,6690.736510,7068.668696,6987.911977,6876.102306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
379,49420,924.0,1042.0,908.0,891.0,932.0,974.0,1038.0,1066.0,1213.0,1352.0,1404.176543,1438.038645,1468.031208,1497.199895,1526.186875
380,49620,2167.0,2151.0,2271.0,2123.0,2166.0,2232.0,2365.0,2561.0,2640.0,2756.0,2800.860261,2857.072979,2880.158976,2907.445922,2919.269555
381,49660,2652.0,2808.0,2982.0,2858.0,2890.0,2848.0,2850.0,3144.0,3319.0,3425.0,3472.429821,3494.525579,3504.681444,3509.370241,3511.531779
382,49700,655.0,648.0,614.0,690.0,656.0,715.0,666.0,782.0,892.0,1005.0,1014.643782,1076.914930,1056.816864,1101.642345,1071.319482


In [7]:
df['2020'] = df['2020'].astype('int')
df['2021'] = df['2021'].astype('int')
df['2022'] = df['2022'].astype('int')
df['2023'] = df['2023'].astype('int')
df['2024'] = df['2024'].astype('int')
df.head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,10180,774.0,818.0,929.0,839.0,978.0,1032.0,1039.0,1053.0,1093.0,1170.0,1190,1238,1273,1314,1353
1,10420,4395.0,4610.0,4379.0,4327.0,4354.0,4639.0,4834.0,5175.0,5511.0,5787.0,5913,6063,6103,6193,6193
2,10500,923.0,1033.0,1060.0,966.0,1107.0,1041.0,1161.0,1183.0,1300.0,1686.0,1749,1811,1872,1934,1995
3,10540,564.0,544.0,537.0,509.0,631.0,608.0,687.0,744.0,843.0,831.0,883,884,909,912,925
4,10580,4825.0,5284.0,5150.0,5352.0,5216.0,5405.0,6050.0,7728.0,8960.0,7627.0,6357,6690,7068,6987,6876


In [8]:
df.sort_values(by='2024').head()

,CBSA,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
57,16180,1924.0,1585.0,1254.0,1140.0,1047.0,940.0,846.0,750.0,840.0,702.0,578,462,350,241,133
88,19180,291.0,290.0,307.0,267.0,269.0,269.0,269.0,290.0,266.0,249.0,262,265,263,262,263
85,19060,376.0,315.0,336.0,364.0,369.0,379.0,343.0,332.0,340.0,326.0,342,353,352,347,346
201,30300,291.0,278.0,288.0,293.0,285.0,310.0,323.0,301.0,318.0,369.0,365,330,342,363,349
108,21300,362.0,377.0,330.0,378.0,333.0,342.0,309.0,355.0,407.0,347.0,349,354,353,353,353


In [9]:
# create new collection in mongo
arima_bus_app_2019_2024 = db.arima_business_app_2019_2024

In [10]:
df_dict = df.to_dict(orient='records')

In [11]:
arima_bus_app_2019_2024.insert_many(df_dict)